# GitHub commit contributions outputter

This notebook uses the GitHub Graph Api to create a list of all the repositories a user has committed code to.

In [ ]:
#r "nuget:Octokit.GraphQL,0.1.6-beta"

In [ ]:
using Octokit.GraphQL;
using static Octokit.GraphQL.Variable;
using System.Diagnostics.CodeAnalysis;
using System.Threading.Tasks;

### Configure the query

- Set `includeOwnRepositories` to `true` to include your own repositories in the result.
- Create a [Personal Access Token](https://docs.github.com/en/github/authenticating-to-github/creating-a-personal-access-token)
- By default the Graph API returns data for a maximum period of a year, update `pastYears` to set how many years back from now the contributions should be retrieved

In [ ]:
const string username = "<Your GitHub Username>";
const int pastYears = 10;
const bool includeOwnRepositories = false;
const string personalAccessToken = "<Your PAT>";

Helper class to store API result

In [ ]:
class Result : IEquatable<Result>
{
	public string Repository {get;set;}
	public string Url {get;set;}

	public bool Equals(Result other)
	{
		return Repository.Equals(other.Repository) && Url.Equals(other.Url);
	}

	public override int GetHashCode()
    {
		int repoHc = Repository == null ? 0 : Repository.GetHashCode();
        int urlHc = Url == null ? 0 : Url.GetHashCode();

		return repoHc ^ urlHc;
	}

	public override string ToString()
	{
		return $"|[{Repository}]({Url})|";
	}
}

In [ ]:
var productInformation = new ProductHeaderValue(username, username);
var connection = new Connection(productInformation, personalAccessToken);
var results = new List<Result>();

In [ ]:
for (var yearOffset = -1; yearOffset > pastYears * -1; --yearOffset)
{
    var query = new Query()
        .User(username)
        .ContributionsCollection(from: DateTimeOffset.Now.AddYears(yearOffset), to: DateTimeOffset.Now.AddYears(yearOffset + 1))
        .CommitContributionsByRepository()
        .Select(cc => new Result
        {
            Repository = cc.Repository.NameWithOwner,
            Url = cc.Repository.Url
        })
        .Compile();

    results.AddRange(await connection.Run(query));
}

### Output

Output the result to the console. Modify the `ToString()` of the `Result` class to change the output to your liking

In [ ]:
Console.Write(string.Join(Environment.NewLine, results
		.Where(r => includeOwnRepositories || !r.Repository.StartsWith(username))
		.Distinct()
		.OrderBy(r => r.Repository)));